**Beam-ის დაყენება; საჭირო იმპორტები**

In [2]:
%pip install --quiet apache-beam

from enum import Enum

import apache_beam as beam
import apache_beam as beam

from apache_beam.dataframe import convert
from apache_beam.dataframe.io import read_csv, to_csv

import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

# თუ ეს error-ები და warning-ები გექნებათ, შეგიძლიათ დააიგნოროთ: 
# ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.
# ან
# WARNING:apache_beam.runners.interactive.interactive_environment:Dependencies required for Interactive Beam PCollection visualization are not available, please use: `pip install apache-beam[interactive]` to install necessary dependencies to enable all data visualization features.
# WARNING:root:Make sure that locally built Python SDK docker image has Python 3.7 interpreter.


Note: you may need to restart the kernel to use updated packages.


ERROR: 
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\DataEng\lib\site-packages\apache_beam\io\gcp\bigquery.py", line 341, in <module>
    import google.cloud.bigquery_storage_v1 as bq_storage
ModuleNotFoundError: No module named 'google.cloud.bigquery_storage_v1'


In [3]:
pip show apache_beam

Name: apache-beam
Version: 2.34.0
Summary: Apache Beam SDK for Python
Home-page: https://beam.apache.org
Author: Apache Software Foundation
Author-email: dev@beam.apache.org
License: Apache License, Version 2.0
Location: c:\programdata\anaconda3\envs\dataeng\lib\site-packages
Requires: avro-python3, crcmod, dill, fastavro, future, grpcio, hdfs, httplib2, numpy, oauth2client, orjson, protobuf, pyarrow, pydot, pymongo, python-dateutil, pytz, requests, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# **ამოცანები**

**ამოცანა #1 - სტუდენტების საბოლოო ნიშნების დათვლა**
*(1 ქულა)*

ამ ამოცანის მიზანია, რამოდენიმე სტუდენტისთვის ერთ-ერთი საგნის ფინალური ქულა დავთვალით და გავიგოთ, ჩააბარა თუ არა სტუდენტმა ეს საგანი. კრიტერიუმები მარტივია და სულ ორი კომპონენტია:

 1) შუალედური გამოცდა, რომლის ქულებიც თითოეული სტუდენტისთვის მოცემულია midterm_grades ცვლადში. ეს არის tuple-ების მასივი. tuple-ის პირველი ელემენტია სტუდენტის სახელი, მეორე - გამოცდაში მიღებული ქულა;
 
 2) ფინალური პროექტი, რომლის ქულებიც თითოეული სტუდენტისთვის მოცემულია project_grades ცვლადში. ესეც არის tuple-ების მასივი. 

საგნის ჩააბარების კრიტერიუმი: თუ სტუდენტის საშუალო ქულა 50.0-ია, მაშინ ვთვლით, რომ სტუდენტმა საგანი ჩააბარა.

ამოცანაში უნდა დაითვალოთ სტუდენტების საშუალო ქულა ორივე კომპონენტში და საბოლოო შედეგი წარმოადგინოთ pcollection-ის სახით. ამ კოლექციის თითოეული ელემენტი უნდა იყოს დაფორმატებული ასეთნაირად:

*Student1 has passed the subject with avg grade of 100.0*




---



ამოცანა უნდა შეასრულოთ Beam-ის გამოყენებით. ამისთვის გაწვდით მინიმალისტურ ამოხსნას (უფრო სტრუქტურას), რომელიც თქვენ უნდა გაარჩიოთ და დაასრულოთ. გაითვალისწინეთ:
 * ამოხსნის ძირითადი ჩონჩხი უკვე აწყობილია. თქვენ საწყისი pcollection-ები სწორად უნდა დააგენერიროთ და დაასრულოთ apply_transforms ფუნქციის იმპლემენტაცია. apply_transforms არის ის ადგილი, სადაც ყველა მოთხოვნილი ტრანფორმაცია ხდება pcollection-ებზე.
 * ფორმატირებისთვის უნდა დაიხმაროთ კლასი StudentGrade. რა თქმა უნდა, მისი იმპლემენტაციაც თქვენი მოსაფიქრებელია. 🐱‍💻

In [4]:
midterm_grades = [('Student1', 100.0),('Student2', 50.0),('Student3', 75.0)]
project_grades = [('Student1', 100.0),('Student2', 50.0),('Student4', 90.0),('Student5', 99.0)]

PASSING_GRADE = 50.0

class StudentGrade:
  
    def __init__(self, student_name, midterm_grade, project_grade):
        self.student_name = student_name
        self.midterm_grade = midterm_grade
        self.project_grade = project_grade
        self.avg_grade = (midterm_grade + project_grade) / 2.0

    def __str__(self):
        res = self.student_name + ' has'
        res += ' failed' if self.avg_grade < 50.0 \
                else ' passed'
        res += ' the subject with avg grade of ' + str(self.avg_grade)
        return res

def apply_transforms(midterm_grades, project_grades):
    def createStudent(cgbk):
        student_name, grades = cgbk
        if not grades['Midterm']: grades['Midterm'].append(0.0)
        if not grades['Project']: grades['Project'].append(0.0)
        return StudentGrade(student_name, grades['Midterm'][0], grades['Project'][0])
    
    grouped = ({'Midterm': midterm_grades, 'Project': project_grades} | beam.CoGroupByKey())
    return (grouped | beam.Map(createStudent))

with beam.Pipeline() as p:
    
  midterms = p | 'Midterm Grades' >> beam.Create(midterm_grades)
  projects = p | 'Project Grades' >> beam.Create(project_grades)
  (
      apply_transforms(midterms, projects)
      | beam.Map(print)
  )

Student1 has passed the subject with avg grade of 100.0
Student2 has passed the subject with avg grade of 50.0
Student4 has failed the subject with avg grade of 45.0
Student5 has failed the subject with avg grade of 49.5
Student3 has failed the subject with avg grade of 37.5


**ამოცანა #2 - ისევ და ისევ Marvel and DC: the good and the bad**
*(2 ქულა)*


უნდა ვიპოვოთ Top 10 ყველაზე ძლევამოსილი პერსონაჟი Marvel-ისა და DC-ის სამყაროებიდან Beam-ის გამოყენებით.

Dataset არის თქვენთვის უკვე ნაცნობი (ავად თუ კარგად 😁). სულ გვაქვს ორი csv ფაილი:
* characters_info.csv - პერსონაჟების შესახებ მონაცემები (სახელი, კეთილია/ბოროტია, სქესი, ფიზიკური აღწერილობა, რომელი სამყაროს ნაწილია და ა. შ. ველები: Name, Alignment, Gender, EyeColor, Race, HairColor,	Publisher,	SkinColor,	Height, Weight;
* character_stats.csv - სხვადასხვა პერსონაჟის სტატისტიკური მონაცემები ველებით Name, Alignment, Intelligence, Strength, Speed, Durability, Power, Combat, Total (ჯამური ქულა ყველა კომპონენტში);

დუბლირებებზე შეგიძლიათ არ იდარდოთ. ასევე, რა პერსონაჟებიც არიან ერთ ფაილში, იგივე პერსონაჟებია მეორეშიც და პირიქით. ფაილებში პერსონაჟები არა მარტო მარველის ან dc-ის, არამედ სხვა სამყაროებიდანაცაა (Tolkien, Harry Potter...) .

რომ დავაკონკრეტოთ ამოცანა:
  * უნდა მოვძებნოთ Top 10 ყველაზე ძლევამოსილი პერსონაჟები მხოლოდ და მხოლოდ Marvel-ის ან DC-ის სამყაროებიდან. რაც უფრო მაღალია Total ველის მნიშვნელობა (stats ფაილი), მით უფრო ძლევამოსილია პერსონაჟი;
  * გვაინტერესებს Top 10 ყველაზე ძლევამოსილი პერსონაჟი როგორც კეთილ გმირებში (alignment = 'good'), ასევე ბოროტებში (alignment = 'bad') **ცალ-ცალკე**. ანუ ტოპ 10 ყველაზე ძლიერი, კეთილი პერსონაჟი და იგივე ბოროტებისთვისაც. ყველა სხვა alignment არ გვაინტერესებს;
  * საბოლოო შედეგში გვინდა, რომ გვქონდეს მხოლოდ შემდეგი ველები: name, score (იგივე, რაც Total ველი), gender, race, universe. ეს შედეგები თითოეული aligment-ისთვის უნდა შევინახოთ ცალ-ცალკე ფაილში: top_10_bad_heroes.csv და top_10_good_heroes.csv. ანუ სულ ეს ორი output ფაილი გვექნება. ათეული დალაგებული უნდა იყოს score-ის კლებადობის მიხედვით.


**ტექნიკური დეტალები:**
 
 ამოხსნა მთლიანად თქვენს ფანტაზიაზეა, მაგრამ გარკვეულ რჩევებს მაინც მოგცემთ:
 * ამ ამოცანას უფრო იოლად ამოხსნით, თუ dataframe-ებს გამოიყენებთ;
 * იმისთვის, რომ dataframe-ები ვიზუალურად ნახოთ, დაგჭირდებათ Interactive Beam და მისი Runner, რომელთა შესაბამისი იმპორტებიც უკვე უზრუნველყოფილია. ***ib.collect(df)***-ით შეძლებთ dataframe ნახოთ საჭიროების შემთხვევაში. Pipeline-ის დასაწყისი დატოვეთ ისე, როგორც მოგაწოდეთ;
 * Beam-ის DataFrame-ები Pandas-ისას სინტაქსურად ძალიან ჰგავს, ოღონდ გარკვეული შეზღუდვებიც აქვს. მაგალითად, ინდექსების გარეშე Beam-ს პრობლემები აქვს ზოგ სიტუაციაში (თუნდაც merge-ისას) და ამის გამო გარკვეული exception-ები შეიძლება გქონდეთ. ამიტომ ინდექსები გაითვალისწინეთ;
 * მნიშვნელოვანი საკითხი: Beam-ის Pipeline-ში არც ერთი ოპერაციის label არ უნდა მეორდებოდეს. თუ რაღაც მიზეზებით ერთი და იგივე ტრანსფორმაცია ორჯერ გაქვთ ფაიფლაინში და სახელი საერთოდ არ მიუთითეთ, default-ად ერთსა და იმავე სახელს მიანიჭებს Beam და გექნებათ exception. ამიტომ გაითვალისწინეთ, რომ სხვადასხვა label უნდა მიუწეროთ ყველა ტრანსფორმაციას. Label-ები ორნაირად შეიძლება მიეთითოს, სიტუაციიდან გამომდინარე. მაგალითად: **pipeline | 'ლეიბლის უნიკალური სახელი' >> ტრანსფორმაცია**  ან **beam.Map(..., label='უნიკალური სახელი')**;
 * class Alignment(Enum): უბრალოდ enum-ია, სადაც alignment-ები გვაქვს გადანომრილი. აუცილებელი არაა გამოყენება, სურვილისამებრ;
 * Top 10 პერსონაჟის საპოვნელად, სავარაუდოდ, დაგჭირდებათ beam.transforms.combiners მოდულის ფუნქციები. თუ სხვა იდეა არ გექნებათ, მაგალითად, შეგიძლიათ ნახოთ აი, ეს ფუნქცია: [Top.Of](https://beam.apache.org/releases/pydoc/2.2.0/apache_beam.transforms.combiners.html#apache_beam.transforms.combiners.Top). [სხვა მაგალითებიც](http://beam.apache.org/documentation/transforms/python/aggregation/top/)
 * აუცილებელი არაა, მაგრამ ყოველი შემთხვევისთვის: თუ თქვენი solution-ის რაღაც მომენტში გექნებათ სიტუაცია, სადაც pipeline-ში გაქვთ pcollection სიის სახით და გინდათ, მიიღოთ სათითაოდ ყველა ელემენტი ამ სიაში, შეგიძლიათ, გამოიყენოთ ჩვენ მიერ მოწოდებული პაწაწინა კლასი BreakList;

 

In [ ]:
class Alignment(Enum):
  bad = 0 
  good = 1
  neutral = 2
  undefined = 3

class BreakList(beam.DoFn):
  def process(self, element):
    for e in element:
      yield e


with beam.Pipeline(InteractiveRunner()) as pipeline:
    characters_info_df = pipeline | 'Read characters info CSV' >> beam.dataframe.io.read_csv('./characters_info.csv')
    characters_stats_df = pipeline | 'Read character stats CSV' >> beam.dataframe.io.read_csv('./character_stats.csv')
    
    characters_stats_total_df = characters_stats_df.loc[:, ['Name', 'Total']]
    characters_stats_total_df['Name1'] = characters_stats_total_df['Name']
    characters_info_pruned_df = characters_info_df.loc[:, ['Name', 'Alignment', 'Gender', 'Race', 'Publisher']]
    
    merged = characters_stats_total_df.set_index('Name1').merge(characters_info_pruned_df.set_index('Name'), left_index=True, right_index=True)
    
    merged_marvel = merged[merged['Publisher'] == 'Marvel Comics']
    marvel_good = merged_marvel[merged_marvel['Alignment'] == 'good']
    marvel_bad = merged_marvel[merged_marvel['Alignment'] == 'bad']
    
    marvel_good_top10 = (convert.to_pcollection(marvel_good)
                         | 'Get top 10 good by total' >> beam.combiners.Top.Of(10, key=lambda row: row[1], reverse=False)
                         | 'Break list top 10 good' >> beam.ParDo(BreakList()))
    marvel_good_top10 = marvel_good_top10 | 'Write out schema for top 10 good' >> beam.Map(
                             lambda row: beam.Row(
                                 name = row[0],
                                 score = row[1],
                                 gender = row[3],
                                 race = row[4],
                                 universe = row[5]
                             ))
    
    marvel_bad_top10 = (convert.to_pcollection(marvel_bad)
                         | 'Get top 10 bad by total' >> beam.combiners.Top.Of(10, key=lambda row: row[1], reverse=False)
                         | 'Break list top 10 bad' >> beam.ParDo(BreakList()))
    marvel_bad_top10 = (marvel_bad_top10 | 'Write out schema for top 10 bad' >> beam.Map(
                             lambda row: beam.Row(
                                 name = row[0],
                                 score = row[1],
                                 gender = row[3],
                                 race = row[4],
                                 universe = row[5]
                             ))
                        | beam.Reshuffle())
    
    marvel_good_top10_df = convert.to_dataframe(marvel_good_top10)
    marvel_good_top10_df = marvel_good_top10_df.set_index('name')
    marvel_good_top10_df.to_csv('./top_10_good_heroes')
    
    marvel_bad_top10_df = convert.to_dataframe(marvel_bad_top10)
    marvel_bad_top10_df = marvel_bad_top10_df.set_index('name')
    marvel_bad_top10_df.to_csv('./top_10_bad_heroes')

**ამოცანა #3 - Breaking News**
*(3 ქულა)*

დატასეტში bbc_news.csv მოცემული გვაქვს 1500-მდე მოკლე სტატია BBC News-იდან. თითოეულ სტატიას აქვს უნიკალური ID და კატეგორიის label-ი (tech, business, politics, entertainment, sport).

მიზანია, რომ თითოეულ სტატიას დავუგენერიროთ tag-ები შემდეგი მარტივი კრიტერიუმით:  უნდა მოვძებნოთ 3 ყველაზე ხშირი სიტყვა ამ სტატიაში. აუცილებელია, რომ თითოეული სიტყვა მინიმუმ 10-ჯერ მაინც გვხვდებოდეს კონკრეტულ სტატიაში.

გაითვალისწინეთ, რომ:
 * Tag-ები არ უნდა იყოს ე.წ. stopword-ები ან სიმბოლოები. მათი ჩამონათვალი არის ფაილში stopwords.txt;
 * Tag-ები არ შეიძლება იყოს რიცხვები;
 * Tag-ები არ შეიძლება იყოს ერთ ან ორ სიმბოლოიანი სიტყვები;
 * სიტყვებად ჩავთვალოთ შემდეგი სიმბოლოებით დაყოფილი ტოკენები: 

          whitespace

          ,

          :

          !

          ?

 * ტრანსფორმაციები უნდა შეასრულოთ, რასაკვირველია, Beam-ის გამოყენებით და არა უბრალოდ პითონის ხარჯზე. 😏

საბოლოო resultset უნდა ჩაწეროთ csv ფაილად და უნდა შედგებოდეს ველებისგან: ***ArticleID, Category, Tag, Frequency***


In [ ]:
import re

with beam.Pipeline(InteractiveRunner()) as pipeline:
    news_df = pipeline | 'Read news CSV' >> beam.dataframe.io.read_csv('./bbc_news.csv')
    stopwords_txt = open('./stopwords.txt', 'r')
    stopwords = stopwords_txt.read().split('\n')
    news_pc = convert.to_pcollection(news_df)
    news_filtered = (news_pc | 'Split text' >> beam.Map(lambda row: (row[0], re.split(': |\! |\? |\, | |\. ', row[1]), row[2]))
                     | 'Filter words' >> beam.Map(lambda row: (row[0], row[1] | beam.Filter(lambda x: (x not in stopwords and not (isinstance(x, int) or isinstance(x, float)) and len(x) > 2)), row[2]))
                     | 'Get element counts' >> beam.Map(lambda row: (row[0], row[1] | beam.combiners.Count.PerElement(), row[2]))
                     | 'Filter by count (>=10)' >> beam.Map(lambda row: (row[0], row[1] | beam.Filter(lambda x: x[1] >= 10), row[2]))
                     | 'Get top 3' >> beam.Map(lambda row: (row[0], row[1] | beam.combiners.Top.Of(3, key=lambda a: a[1], reverse=False), row[2]))
                     | 'Filter rows without tags' >> beam.Filter(lambda row: len(row[1][0]) > 0)
                     | 'Remove nested list' >> beam.Map(lambda row: beam.Row(ArticleID=row[0], TagFreq=row[1][0], Category=row[2])))
    news_filtered_df = convert.to_dataframe(news_filtered)
    news_exp_df = news_filtered_df.explode('TagFreq')
    news_exp_pc = convert.to_pcollection(news_exp_df) | 'Separate columns' >> beam.Map(lambda row: beam.Row(ArticleID=row[0], Category=row[2], Tag=row[1][0] if row[1] != '-' else '-', Frequency=row[1][1] if row[1] != '-' else '-'))
    news_final_df = convert.to_dataframe(news_exp_pc)
    news_final_df = news_final_df.set_index('ArticleID')
    news_final_df.to_csv('./resultset')



---


შესრულებული დავალება მოგვაწოდეთ ნოუთბუქის სახით.
წარმატებები!